# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd


In [2]:
#Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [7]:
#Your code here; use a subquery. No join will be necessary.
cur.execute(''' SELECT customerNumber, contactLastName,
                        contactFirstName 
                FROM customers 
                where customerNumber in 
                (SELECT customerNumber 
                FROM orders
                WHERE orderDate = '2003-01-31');''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
display(df.head())

cur.execute('''select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';''')

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [15]:
#Your code here
cur.execute('''SELECT productName, sum(quantityOrdered) as numsold
            FROM orderdetails 
            JOIN products using (productCode)
            GROUP BY productName
            ORDER BY numsold DESC''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,productName,numsold
0,1992 Ferrari 360 Spider red,1808
1,1937 Lincoln Berline,1111
2,American Airlines: MD-11S,1085
3,1941 Chevrolet Special Deluxe Cabriolet,1076
4,1930 Buick Marquette Phaeton,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [17]:
#Your code here
cur.execute('''SELECT productName, count(distinct orderNumber) as numordered
            FROM orderdetails 
            JOIN products using (productCode)
            GROUP BY productName
            ORDER BY numordered DESC''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()


,productName,numordered
0,1992 Ferrari 360 Spider red,53
1,18th Century Vintage Horse Carriage,28
2,1900s Vintage Bi-Plane,28
3,1900s Vintage Tri-Plane,28
4,1913 Ford Model T Speedster,28


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [47]:
#Your code here
cur.execute("""select employeeNumber,
                    officeCode, offices.city, firstName, lastName
                    from employees 
                    join offices 
                    using(officeCode)
                    join customers 
                    on employees.employeeNumber = customers.salesRepEmployeeNumber
                    join orders
                    using (customerNumber)
                    join orderdetails
                    using (orderNumber)
                    where productCode in 
                    (select productCode from products
                    join orderdetails using (productCode)
                    join orders using (orderNumber)
                    group by productCode
                    having count(customerNumber) <= 20 );""")
                
           


df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [46]:
#Your code here
cur.execute("""select employeeNumber, firstName, lastName,
                COUNT(customerName) 
                from employees 
                join customers
                on employees.employeeNumber = customers.salesRepEmployeeNumber
                GROUP BY employeeNumber
                Having AVG(creditLimit) >= 15000;""")
                
           


df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,firstName,lastName,COUNT(customerName)
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!